API调用

In [14]:
import getpass
import os

os.environ["TOGETHER_API_KEY"] = "api_key"

from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    base_url="url",
    api_key=os.environ["TOGETHER_API_KEY"],
    model="model_name",
)


简单测试API调用是否成功

In [18]:
from langchain.schema import HumanMessage
from langchain.prompts import PromptTemplate
template = '''请回答：{input}'''
template2 = PromptTemplate(template=template,input_variables=['input'])
print(model([HumanMessage(content = template2.format(input='1+1'))]).content)

在数学中，$1+1 = 2$。

但在一些特殊情境或特定的概念理解中，可能有其他特殊的含义或解释哦。你是想问关于这个的什么呢？可以进一步说明。


Finance_Agent Demo（双模型串联）

In [19]:
from langchain.schema import HumanMessage
from langchain.prompts import PromptTemplate

# 定义三个模型的PromptTemplate
template1 = PromptTemplate(
    template='''
    # 角色
    你是一位资深的财务分析师，善于通过企业的主要财务指标表、资产负债表来分析企业的偿债能力。

    # 目标
    通过提供的主要财务指标表、资产负债表，结合你的金融领域知识，计算用于评估企业偿债能力的财务指标，并与前两年的数据进行对比，从而揭示出企业的风险点所在。

    # 技能
    - 丰富的股票市场知识
    - 熟练的财务分析技能
    - 对财表隐藏风险点的洞察能力
    - 熟练的python数据分析能力
    # 工作流程
    1.接收财表数据和领域知识
    2.根据实际情况，逐步规划你的财务分析步骤，每个步骤应该详尽、科学严谨、可操作性强
    3.根据你规划出的财务分析步骤，使用python编程语言进行分析
    4.整理数据分析结果，输出你的财务分析步骤和计算出的用于企业偿债能力分析的财务指标。

    # 限制
    仅输出你的财务分析步骤和计算出的用于企业偿债能力分析的财务指标。

    # 输入参数
    主要指标表：{input}
    ''',
    input_variables=['input']
)

template2 = PromptTemplate(
    template='''
    # 角色
    你是一位资深的财务分析师，善于通过企业的主要财务指标表、资产负债表来分析企业的偿债能力。

    # 目标
    通过提供的主要财务指标表、资产负债表，结合你的金融领域知识，计算用于评估企业偿债能力的财务指标，并与前两年的数据进行对比，从而揭示出企业的风险点所在。

    # 技能
    - 丰富的股票市场知识
    - 熟练的财务分析技能
    - 对财表隐藏风险点的洞察能力
    - 熟练的python数据分析能力

    # 工作流程
    1.接收财务指标计算结果和领域知识
    2.结合计算出的指标和你的知识，揭露企业在偿债能力上可能出现的风险点。

    # 限制
    仅输出该企业在偿债能力上可能出现的风险点

    请解释：{input}
    ''',
    input_variables=['input']
)

# template3 = PromptTemplate(
#     template='''
#     # 技能
#     1. 接收财务指标计算结果和领域知识
#     2. 结合计算出的指标和你的知识，揭露企业在偿债能力上可能出现的风险点。

#     # 限制
#     仅输出该企业在偿债能力上可能出现的风险点。

#     请总结：{input}
#     ''',
#     input_variables=['input']
# )

def model1(input):
    return model([HumanMessage(content=template1.format(input=input))]).content

def model2(input):
    return model([HumanMessage(content=template2.format(input=input))]).content

# def model3(input):
#     return model([HumanMessage(content=template3.format(input=input))]).content

def workflow(input):
    result1 = model1(input)
    final_result = model2(result1)
    # final_result = model3(result2)
    return final_result

# 使用工作流
input_data = '科目\时间	2023	2022	2021	2020	2019	2018	2017	2016	2015	2014	2013	2012	2011	2010	2009	2008	2007	2006	2005	2004	2003	2002	2001	2000	1999	1998	1997	1996	1995	1994	1993	1992报表核心指标(元)	*所有者权益（或股东权益）合计(元)	145154393200	164262516700	163882892400	155273877400	131392301000	108925851800	93451834400	77476124600	71832939000	61231160400	2201461400	2200675300	2112188900	1978299100	1858386400	1581294100	1758320500	1283924600	1234424700	1191326400	1114315300	1049073100	343312000	269312000	252884000	175398900	140958500	150744500	164436400	158704800	156277200	115307200*资产合计(元)	1193922084900	1365382105100	1469097908700	1397336286400	1145706529900	1036545456300	848532814700	733137955500	600436070400	508958661500	6158879500	5966315500	5623835500	4465521300	3857513800	2932135600	2906985800	2614355600	2738544900	2811925600	2338788200	2219474300	1450903000	1074244500	785563600	552345700	435481400	439032600	502545400	484953100	436279600	363045700*负债合计(元)	1048767691700	1201119588300	1305215016400	1242062409000	1014314228900	927619604500	755080980300	655661830900	528603131500	447727501100	3957418100	3765640100	3511646700	2487222200	1999127400	1350841400	1148665400	1330431000	1504120200	1620599200	1224472900	1170401100	1107590900	804932600	532679600	376946800	294523000	288288100	338109000	326248400	280002300	63863500*归属于母公司所有者权益合计(元)	80339661500	90648914800	89972496100	84776400800	78901190500	70104306300	62529251600'
output = workflow(input_data)
print(output)


企业偿债能力可能存在的风险点：
- 资产负债率整体呈波动上升趋势，尤其是近两年上升较快，反映企业负债水平较高，长期偿债压力较大，可能面临较高的财务风险，如资金链紧张、偿债能力不足引发违约等风险。
- 流动比率虽有一定波动，但多数年份低于正常水平，尤其是近两年明显偏低，表明企业流动资产对流动负债的覆盖能力较弱，短期偿债能力较弱，可能存在无法及时偿还短期债务的风险，易引发流动性危机。
- 速动比率情况与流动比率类似，多数年份不理想，尤其近两年表现较差，企业的速动资产（扣除存货等较难快速变现资产）对流动负债的保障程度不足，短期偿债风险较高。
- 从多年数据变化趋势看，企业偿债能力呈现逐步减弱的态势，需密切关注其经营状况和财务状况的变化，以防偿债能力持续恶化导致财务困境的出现。


RAG部分

Embedding（嵌入生成）：
嵌入生成是将文本数据转换为高维向量表示的过程。这个过程通常通过预训练的语言模型（如 BERT, RoBERTa, 或 Sentence-BERT）来完成。
每个文本输入（如句子、段落或文档）都会被编码成一个固定大小的向量，这些向量捕捉了文本的语义信息。

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')
sentences = ["This is an example sentence", "Each sentence is converted"]
embeddings = model.encode(sentences)

向量匹配（Vector Matching）：
向量匹配是通过计算向量之间的相似度来检索相关信息的过程。通常使用的相似度度量是余弦相似度或欧氏距离。
Faiss 是一个高效的相似度搜索库，用于处理大规模向量搜索。

In [ ]:
import faiss
import numpy as np

# 假设 embeddings 是一个包含文本向量的 numpy 数组
d = embeddings.shape[1]
index = faiss.IndexFlatL2(d)  # 使用欧氏距离
index.add(embeddings)
D, I = index.search(embeddings, k=5)  # 检索前 5 个相似向量


检索增强生成（RAG）：
RAG 系统通过检索相关信息来增强生成模型的输出。这个过程包括使用嵌入模型生成查询的向量，然后在向量数据库中搜索最相似的向量，最后将检索到的信息传递给生成模型以生成上下文相关的响应。

In [ ]:
def retrieve_documents(query, model, index):
    query_embedding = model.encode([query])
    D, I = index.search(query_embedding, k=5)
    return [documents[i] for i in I[0]]

def generate_response(query, model, index, generation_model):
    relevant_docs = retrieve_documents(query, model, index)
    context = " ".join(relevant_docs)
    prompt = f"Context: {context}\n\nQuery: {query}\nResponse:"
    response = generation_model.generate(prompt)
    return response


简单测试代码

In [ ]:
# 导入所需的库
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# 步骤 1：嵌入生成
# 加载 SentenceTransformer 模型
model = SentenceTransformer('all-MiniLM-L6-v2')

# 定义文档列表，这些是需要嵌入和检索的文档
documents = ["Document 1 text", "Document 2 text", "Document 3 text"]

# 使用模型对文档进行编码，生成文档的嵌入向量
embeddings = model.encode(documents)

# 步骤 2：构建向量索引
# 获取嵌入向量的维度
d = embeddings.shape[1]

# 创建一个 L2 距离的 FAISS 索引
index = faiss.IndexFlatL2(d)

# 将嵌入向量添加到索引中
index.add(np.array(embeddings, dtype=np.float32))

# 步骤 3：检索和生成
# 定义一个函数，用于根据查询从索引中检索相关文档
def retrieve_documents(query, model, index, documents):
    # 对查询进行编码，生成查询的嵌入向量
    query_embedding = model.encode([query])
    
    # 在索引中搜索与查询最相似的文档，k=3 表示返回 3 个最相似的文档
    D, I = index.search(np.array(query_embedding, dtype=np.float32), k=3)
    
    # 根据索引结果返回相关文档
    return [documents[i] for i in I[0]]

# 定义一个函数，用于生成响应
def generate_response(query, model, index, documents):
    # 使用检索函数获取与查询相关的文档
    relevant_docs = retrieve_documents(query, model, index, documents)
    
    # 将相关文档连接成一个上下文字符串
    context = " ".join(relevant_docs)
    
    # 构建提示信息，包括上下文和查询
    prompt = f"Context: {context}\n\nQuery: {query}\nResponse:"
    
    # 生成响应，简单地将上下文和查询信息包含在内
    response = f"Based on the context: {context}, the answer to '{query}' is ..."
    
    # 返回生成的响应
    return response

# 使用示例
# 定义一个查询
query = "What is the main point of Document 1?"

# 使用生成响应函数根据查询生成响应
response = generate_response(query, model, index, documents)

# 输出响应
print(response)
